# Chapter 3: Resolution of Linear Systems
## 01 - The Gaussian Elimination Method

---

### 1. Introduction

A system of linear equations is a collection of first-degree equations. Geometrically, the solution to a linear system represents the intersection point (or set of points) of lines, planes, or hyperplanes.

These systems appear in nearly every field of science and engineering, from analyzing electrical circuits and mechanical structures to image processing and economic modeling.

A system with $m$ equations and $n$ unknowns can be written in matrix form as:
$$ Ax = b $$
Where:
- $A$ is the coefficient matrix ($m \times n$).
- $x$ is the vector of unknowns ($n \times 1$).
- $b$ is the vector of independent terms ($m \times 1$).

### 2. Existence and Uniqueness of a Solution

A linear system can have:
1.  **A single unique solution**: The system is called **consistent and determined**.
2.  **Infinite solutions**: The system is **consistent and undetermined**.
3.  **No solution**: The system is **inconsistent**.

For square systems (where the number of equations equals the number of unknowns), a condition for the existence of a unique solution is that the coefficient matrix $A$ must be **non-singular**, which is equivalent to its **determinant being non-zero** (det(A) ≠ 0). If the determinant is zero, the system will have either infinite solutions or no solution.

> **Note**: Checking the determinant is only practical for small systems. For larger systems, the Gaussian Elimination method allows us to determine the nature of the solution more efficiently.

### 3. The Gaussian Elimination Method (Without Pivoting)

The goal of Gaussian Elimination is to transform a complex linear system into an equivalent **upper triangular system**, which is easy to solve.

$$ 
\begin{pmatrix}
a_{11} & a_{12} & a_{13} \\
a_{21} & a_{22} & a_{23} \\
a_{31} & a_{32} & a_{33}
\end{pmatrix}
\begin{pmatrix} x_1 \\ x_2 \\ x_3 \end{pmatrix} = 
\begin{pmatrix} b_1 \\ b_2 \\ b_3 \end{pmatrix}
\quad \xrightarrow{\text{Elimination}} \quad
\begin{pmatrix}
u_{11} & u_{12} & u_{13} \\
0 & u_{22} & u_{23} \\
0 & 0 & u_{33}
\end{pmatrix}
\begin{pmatrix} x_1 \\ x_2 \\ x_3 \end{pmatrix} = 
\begin{pmatrix} c_1 \\ c_2 \\ c_3 \end{pmatrix}
$$

The process has two phases:
1.  **Elimination Phase (Forward Elimination)**: Systematically zero out the elements below the main diagonal of the matrix.
2.  **Back Substitution Phase**: Solve the resulting triangular system, starting from the last equation and working upwards.

#### Algorithm: Elimination Phase

1.  **Form the augmented matrix**: $[A|b]$.
2.  For each column $j$ from $1$ to $n-1$:

    a. The element $a_{jj}$ is the **pivot** for the step.

    b. For each row $i$ below the pivot (from $j+1$ to $m$):
        i. Calculate the multiplier: $m_{ij} = \frac{a_{ij}}{a_{jj}}$.
        ii. Replace row $L_i$ with $L_i - m_{ij} \cdot L_j$. This will zero out the element $a_{ij}$.

#### Algorithm: Back Substitution

After elimination, the system is in the form $Ux = c$. The last equation will have only one unknown, $x_n$.

1.  Solve for $x_n$ from the last equation:
    $$ x_n = \frac{c_n}{u_{nn}} $$
2.  For $i$ from $n-1$ down to $1$:
    $$ x_i = \frac{1}{u_{ii}} \left( c_i - \sum_{j=i+1}^{n} u_{ij} x_j \right) $$

### 4. The Need for Pivoting

The naive method presented above will fail in two scenarios:

1.  **Zero Pivot**: If the pivot element $a_{jj}$ is zero, the calculation of the multiplier $m_{ij} = a_{ij}/a_{jj}$ will result in a division by zero.
2.  **Small Pivot**: If the pivot is a very small number compared to other elements in its column, it can cause **numerical instability**. Small round-off errors during computation are magnified, leading to inaccurate or completely wrong results.

The solution to both these problems is the **partial pivoting** strategy.

### 5. Gaussian Elimination with Partial Pivoting

The strategy is simple but highly effective:

> Before starting the elimination for column $j$, find the element with the **largest absolute value** in column $j$, from the main diagonal downwards. Swap the row containing this element with the current pivot row.

This ensures that we always divide by the largest possible number, which minimizes the propagation of round-off errors and avoids division by zero (unless the entire sub-column is zero, which indicates a singular matrix).

In [2]:
import numpy as np

def gaussian_elimination_pivot(A, b):
    """Solves a linear system Ax=b using Gaussian Elimination with partial pivoting."""
    n = len(b)
    Ab = np.hstack([A.astype(float), b.astype(float).reshape(-1, 1)])
    print("Initial Augmented Matrix:\n", Ab)

    # Elimination Phase
    for j in range(n):
        # Partial Pivoting
        # Find the row index with the largest absolute pivot value
        max_row_index = np.argmax(np.abs(Ab[j:, j])) + j
        # Swap the current row with the row with the largest pivot
        if max_row_index != j:
            print(f"\nPivoting: Swapping row {j+1} with {max_row_index+1}")
            Ab[[j, max_row_index]] = Ab[[max_row_index, j]]
            print("Matrix after swap:\n", Ab)
        
        pivot = Ab[j, j]
        if abs(pivot) < 1e-12: # If the largest possible pivot is zero
            raise ValueError("Matrix is singular. System may have no or infinite solutions.")
        
        for i in range(j + 1, n):
            multiplier = Ab[i, j] / pivot
            Ab[i, :] = Ab[i, :] - multiplier * Ab[j, :]
        print(f"\nAfter elimination step {j+1}:\n", Ab)

    # Back Substitution Phase
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (Ab[i, -1] - np.dot(Ab[i, i+1:n], x[i+1:n])) / Ab[i, i]
    
    return x

### 6. Example of Numerical Instability

Let's solve a system that is specifically designed to fail without pivoting when using limited precision. Consider the system:

$$ \begin{cases} 0.001x_1 + 1.0x_2 = 1.0 \\ 1.0x_1 + 1.0x_2 = 2.0 \end{cases} $$

The exact solution is $x_1 = \frac{1000}{999} \approx 1.001$ and $x_2 = \frac{998}{999} \approx 0.999$.

Now, let's solve this manually using **3 significant digits** for all calculations.

#### Case 1: Without Pivoting
The pivot is `0.001`.
1.  **Multiplier**: $m_{21} = \frac{1.0}{0.001} = 1000$.
2.  **New Row 2**: $L_2 = L_2 - 1000 \cdot L_1$
    - New $a_{22} = 1.0 - 1000 \cdot 1.0 = -999$.
    - New $b_2 = 2.0 - 1000 \cdot 1.0 = -998$.
3.  **Triangular System**:
    $$ \begin{cases} 0.001x_1 + 1.0x_2 = 1.0 \\ -999x_2 = -998 \end{cases} $$
4.  **Back Substitution**:
    - $x_2 = \frac{-998}{-999} \approx 0.999$.
    - $0.001x_1 + 1.0(0.999) = 1.0 \implies 0.001x_1 = 0.001 \implies x_1 = 1.0$.

The calculated solution is $[1.0, 0.999]$, which seems good. However, if the right-hand side were slightly different, e.g., $b_1=1.01$, the subtraction $1.01 - 1.0(1.009)$ would result in a massive loss of precision. The issue with a small pivot is this catastrophic cancellation.

#### Case 2: With Partial Pivoting
1.  **Pivot Search**: In the first column, $|1.0| > |0.001|$, so we must swap the rows.
2.  **Pivoted System**:
    $$ \begin{cases} 1.0x_1 + 1.0x_2 = 2.0 \\ 0.001x_1 + 1.0x_2 = 1.0 \end{cases} $$
3.  **Multiplier**: $m_{21} = \frac{0.001}{1.0} = 0.001$.
4.  **New Row 2**: $L_2 = L_2 - 0.001 \cdot L_1$
    - New $a_{22} = 1.0 - 0.001 \cdot 1.0 = 0.999$.
    - New $b_2 = 1.0 - 0.001 \cdot 2.0 = 1.0 - 0.002 = 0.998$.
5.  **Triangular System**:
    $$ \begin{cases} 1.0x_1 + 1.0x_2 = 2.0 \\ 0.999x_2 = 0.998 \end{cases} $$
6.  **Back Substitution**:
    - $x_2 = \frac{0.998}{0.999} \approx 0.999$.
    - $1.0x_1 + 1.0(0.999) = 2.0 \implies 1.0x_1 = 1.001 \implies x_1 = 1.001$.

The solution is $[1.001, 0.999]$, which is much more accurate. By choosing the larger pivot, we avoided multiplying by a large number (1000), which prevented the loss of precision.


### 7. Exercises

#### Exercise 1: Solve the system using Gaussian Elimination
$$ \begin{cases} 2x_1+2x_2+x_3+x_4 = 7 \\ x_1-x_2+2x_3-x_4 = 1 \\ 3x_1+2x_2-3x_3-2x_4 = 4 \\ 4x_1+3x_2+2x_3+x_4 = 12 \end{cases} $$

In [3]:
A1 = np.array([[2, 2, 1, 1],
               [1, -1, 2, -1],
               [3, 2, -3, -2],
               [4, 3, 2, 1]])

b1 = np.array([7, 1, 4, 12])

print("--- Solving Exercise 1 with Pivoting ---")
try:
    solution1 = gaussian_elimination_pivot(A1.copy(), b1.copy())
    print("\nFinal solution x:", solution1)
    # Verification
    print("Verification (A @ x):", A1 @ solution1)
except ValueError as e:
    print(e)

--- Solving Exercise 1 with Pivoting ---
Initial Augmented Matrix:
 [[ 2.  2.  1.  1.  7.]
 [ 1. -1.  2. -1.  1.]
 [ 3.  2. -3. -2.  4.]
 [ 4.  3.  2.  1. 12.]]

Pivoting: Swapping row 1 with 4
Matrix after swap:
 [[ 4.  3.  2.  1. 12.]
 [ 1. -1.  2. -1.  1.]
 [ 3.  2. -3. -2.  4.]
 [ 2.  2.  1.  1.  7.]]

After elimination step 1:
 [[ 4.    3.    2.    1.   12.  ]
 [ 0.   -1.75  1.5  -1.25 -2.  ]
 [ 0.   -0.25 -4.5  -2.75 -5.  ]
 [ 0.    0.5   0.    0.5   1.  ]]

After elimination step 2:
 [[ 4.          3.          2.          1.         12.        ]
 [ 0.         -1.75        1.5        -1.25       -2.        ]
 [ 0.          0.         -4.71428571 -2.57142857 -4.71428571]
 [ 0.          0.          0.42857143  0.14285714  0.42857143]]

After elimination step 3:
 [[ 4.00000000e+00  3.00000000e+00  2.00000000e+00  1.00000000e+00
   1.20000000e+01]
 [ 0.00000000e+00 -1.75000000e+00  1.50000000e+00 -1.25000000e+00
  -2.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -4.71428571e+00 -2.

#### Exercise 2: Analyze the systems regarding the number of solutions

Gaussian Elimination also tells us if a system has infinite solutions or none.
- **No Solution**: If, after elimination, we have a row of the form `[0 0 ... 0 | c]` where `c ≠ 0`, this represents the equation `0 = c`, which is a contradiction.
- **Infinite Solutions**: If we have a row of all zeros `[0 0 ... 0 | 0]`, this indicates a redundant equation and the existence of free variables.

In [ ]:
# Exercise 2a
A2a = np.array([[3, -2, 5, 1],
                [-6, 4, -8, 1],
                [9, -6, 19, 1],
                [6, -4, -6, 15]])
b2a = np.array([7, -9, 23, 11])

print("--- Analyzing System 2a ---")
try:
    solution2a = gaussian_elimination_pivot(A2a.copy(), b2a.copy())
    print("\nFinal solution x:", solution2a)
except ValueError as e:
    print("\nResult:", e)
    
# Exercise 2b
A2b = np.array([[0.252, 0.6, 0.12],
                [0.112, 0.16, 0.24],
                [0.147, 0.21, 0.25]])
b2b = np.array([7, 8, 9])

print("\n--- Analyzing System 2b ---")
try:
    solution2b = gaussian_elimination_pivot(A2b.copy(), b2b.copy())
    print("\nFinal solution x:", solution2b)
except ValueError as e:
    print("\nResult:", e)

--- Analyzing System 2a ---
Initial Augmented Matrix:
 [[ 3. -2.  5.  1.  7.]
 [-6.  4. -8.  1. -9.]
 [ 9. -6. 19.  1. 23.]
 [ 6. -4. -6. 15. 11.]]

Pivoting: Swapping row 1 with 3
Matrix after swap:
 [[ 9. -6. 19.  1. 23.]
 [-6.  4. -8.  1. -9.]
 [ 3. -2.  5.  1.  7.]
 [ 6. -4. -6. 15. 11.]]

After elimination step 1:
 [[  9.          -6.          19.           1.          23.        ]
 [  0.           0.           4.66666667   1.66666667   6.33333333]
 [  0.           0.          -1.33333333   0.66666667  -0.66666667]
 [  0.           0.         -18.66666667  14.33333333  -4.33333333]]

Result: Matrix is singular. System may have no or infinite solutions.

--- Analyzing System 2b ---
Initial Augmented Matrix:
 [[0.252 0.6   0.12  7.   ]
 [0.112 0.16  0.24  8.   ]
 [0.147 0.21  0.25  9.   ]]

After elimination step 1:
 [[ 0.252       0.6         0.12        7.        ]
 [ 0.         -0.10666667  0.18666667  4.88888889]
 [ 0.         -0.14        0.18        4.91666667]]

Pivoting: Swa

### 8. Exercise: Impact of Finite-Precision Arithmetic

**Problem:** Using arithmetic with **two significant digits** for all operations, solve the following linear system using Gaussian Elimination, both with and without partial pivoting. Then, repeat the exercise using two-digit truncation. Discuss your results.

$$
\begin{cases}
16x_1 + 5x_2 = 21 \
3x_1 + 2.5x_2 = 5.5
\end{cases}
$$

The exact solution is ( x_1 = 1 ) and ( x_2 = 1 ).

In [4]:
import math
import numpy as np

def round_sig(x, n_digits):
    """Rounds a number to n significant digits."""
    if x == 0:
        return 0
    return round(x, n_digits - int(math.floor(math.log10(abs(x)))) - 1)

def truncate_sig(x, n_digits):
    """Truncates a number to n significant digits."""
    if x == 0:
        return 0
    step = 10**(math.floor(math.log10(abs(x))) - n_digits + 1)
    return math.trunc(x / step) * step

def solve_finite_precision(A, b, precision_func, n_digits, use_pivot=False):
    """Solves a system applying a precision function at each step."""
    n = len(b)
    Ab = np.hstack([A.astype(float), b.astype(float).reshape(-1, 1)])
    
    for j in range(n):
        if use_pivot:
            max_row = np.argmax(np.abs(Ab[j:, j])) + j
            if max_row != j:
                Ab[[j, max_row]] = Ab[[max_row, j]]
        
        for i in range(j + 1, n):
            multiplier = precision_func(Ab[i, j] / Ab[j, j], n_digits)
            for col in range(j, n + 1):
                term = precision_func(multiplier * Ab[j, col], n_digits)
                Ab[i, col] = precision_func(Ab[i, col] - term, n_digits)

    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        dot_product = 0
        for j in range(i + 1, n):
            term = precision_func(Ab[i, j] * x[j], n_digits)
            dot_product = precision_func(dot_product + term, n_digits)
        numerator = precision_func(Ab[i, -1] - dot_product, n_digits)
        x[i] = precision_func(numerator / Ab[i, i], n_digits)
    return x

# To clearly show the effect of pivoting, we use the version of the system 
# with a small initial pivot.
A_ex = np.array([[3, 2.5], [16, 5]])
b_ex = np.array([5.5, 21])
n_digits = 2

print(f"--- Solving with {n_digits} Significant Digits ---")
sol_round_no_pivot = solve_finite_precision(A_ex.copy(), b_ex.copy(), round_sig, n_digits, use_pivot=False)
print(f"\nRounding without Pivoting: x = {sol_round_no_pivot}")

sol_round_pivot = solve_finite_precision(A_ex.copy(), b_ex.copy(), round_sig, n_digits, use_pivot=True)
print(f"Rounding with Pivoting:    x = {sol_round_pivot}")

sol_trunc_no_pivot = solve_finite_precision(A_ex.copy(), b_ex.copy(), truncate_sig, n_digits, use_pivot=False)
print(f"\nTruncation without Pivoting: x = {sol_trunc_no_pivot}")

sol_trunc_pivot = solve_finite_precision(A_ex.copy(), b_ex.copy(), truncate_sig, n_digits, use_pivot=True)
print(f"Truncation with Pivoting:    x = {sol_trunc_pivot}")

--- Solving with 2 Significant Digits ---

Rounding without Pivoting: x = [1. 1.]
Rounding with Pivoting:    x = [1.   0.94]

Truncation without Pivoting: x = [1. 1.]
Truncation with Pivoting:    x = [1. 1.]


This is a fascinating and counter-intuitive result that provides a deep insight into numerical error. Let's analyze it.

1. **Why did Pivoting with Rounding give a *worse* answer?**

   * **Pivoting was the *correct strategy*.** The algorithm correctly identified that `16` is a better pivot than `3` and swapped the rows.
   * The issue arose from the multiplier calculation: ( m = \frac{3}{16} = 0.1875 ).
   * When this is rounded to **two significant digits**, it becomes `0.19`. This immediate, relatively large rounding error was introduced at the very first step.
   * This initial error then propagated through the subsequent calculations, resulting in the final, less accurate answer of `x = [1.0, 0.94]`.

2. **Why did the *non-pivoted* methods work so well?**

   * Without pivoting, the multiplier was ( $m = \frac{16}{3} \approx 5.333...$ ), which rounds to `5.3`.
   * For this *specific set of numbers*, the errors introduced by using this multiplier and performing the subsequent subtractions happened to cancel each other out almost perfectly.
   * This was a **lucky coincidence**, not a sign of a superior method. It is an unreliable outcome.

3. **Truncation Results:**

   * In this case, truncation also led to the correct answer. This is again due to the specific, simple numbers involved.
   * With only a few operations, the inherent bias of truncation (always making numbers smaller in magnitude) did not have a chance to accumulate and cause a significant final error.

### **Conclusion:**
This exercise demonstrates a crucial concept: **Pivoting is the correct and robust general strategy, even if it doesn't yield a better result in every specific, low-precision case.**

An algorithm designer must protect against the **worst-case scenario**. The non-pivoted method is unstable and would fail dramatically on other problems (like the one in Section 6), whereas the pivoted method is reliable.

This problem is a **pathological case** where the rounding error introduced by the multiplier calculation had a more pronounced effect than the stability gained from choosing a better pivot.